# Figure 2 : Decoding orientation (12 labels)

In [ ]:
%run -n boillerplate.ipynb

# Confusion matrix

In [ ]:
try:
    all_cms = np.load('./data/fig_2_T_cms.npy', allow_pickle = True)
    s_all_cms = np.load('./data/fig_2_T_s_cms.npy', allow_pickle = True)
    all_p_mats = np.load('./data/fig_2_pvals_T_cms.npy', allow_pickle = True)
except:    
    all_cms = get_cm_kfold(shuffled = False)
    s_all_cms = get_cm_kfold(shuffled = True)
    all_p_mats = stats_cm(all_cms, s_all_cms)
    
    np.save('./data/fig_2_T_cms.npy', all_cms)
    np.save('./data/fig_2_T_s_cms.npy', s_all_cms)
    np.save('./data/fig_2_pvals_T_cms.npy', all_p_mats)

In [ ]:
print('Maximum accuracy for narrow bandwidth ', np.max(np.sum(all_cms[0,-2])/all_cms[0,-2].shape[0]))

In [ ]:
print('Maximum accuracy for broad bandwidth ', np.max(np.sum(all_cms[2,-2])/all_cms[2,-2].shape[0]))

In [ ]:
vmax = .9

In [ ]:
fig, axs = plt.subplots(figsize = (12,12), nrows = 3, ncols = 3)

new_cols = colors[[7, 4, 0]]
for ibt, bt in enumerate(cm_bt) :
    for i, t in enumerate(cm_timesteps):
        ax = axs[ibt, i]
        cm = np.sum(all_cms[ibt, i])/all_cms[ibt, i].shape[0]
        pvals = all_p_mats[ibt, i]
        ims = ax.imshow(cm, vmin = 1/N_thetas, vmax=vmax, aspect = 'equal')
        #pval_edges(pvals < .05, ax = ax)
        
        for edge in ['top', 'bottom', 'left', 'right'] :
            ax.spines[edge].set_color(new_cols[ibt])
            ax.spines[edge].set_linewidth(1)  
        
        if ibt == 2 and i == 0:
            ax.set_xlabel(r'$\theta$ pred (°)', fontsize = 18)
            ax.set_ylabel(r'$\theta$ true (°)', fontsize = 18)
            ax.tick_params(axis='x', which='major', pad=15)
            ax.tick_params(axis='both', which='major', labelsize=14)
            ax.set_xticks(np.arange(0, N_thetas))
            ax.set_yticks(np.arange(0, N_thetas))
            ax.set_xticklabels(np.round(thetas*180/np.pi,1), rotation = 45, va = 'center')
            ax.set_yticklabels(np.round(thetas*180/np.pi,1), rotation = 45, va = 'center')
        else :
            ax.set_xticks([])
            ax.set_yticks([])

        if ibt == 2 and i == 2 :
            cax = fig.add_axes([ax.get_position().x1+0.1, # offset from the right of the axis
                                ax.get_position().y0+-.048, #bottom of the colorbar
                                0.02, #width of the colorbar
                                .27]) #height of the colorbar
            cb = fig.colorbar(ims, cax = cax, ticks = np.linspace(1/N_thetas, vmax, 6))
            cb.ax.set_yticklabels(np.round(np.linspace(1/N_thetas, vmax, 6), 2))

            cax.set_ylabel('Classification accuracy', rotation = 270, labelpad = 20,
                          fontsize = 20)
            cb.ax.tick_params(labelsize = 14)

        if i == 0 and ibt == 0:
            ax.text(x = (cm.shape[0] /2)-.3, y = -1.1, s = 't = -150 ms', fontsize = 18,
                   va = 'center', ha = 'center')
        elif i == 1 and ibt == 0:
            ax.text(x = (cm.shape[0] /2)-.3, y = -1.1, s = 't = 0 ms', fontsize = 18,
                   va = 'center', ha = 'center')
        elif i ==2 and ibt == 0:
            ax.text(x = (cm.shape[0] /2)-.3, y = -1.1, s = 't = 300 ms', fontsize = 18,
                   va = 'center', ha = 'center')
            
        ax.autoscale(False)
        ax.set_xlim(-.5, 11.5)
        ax.set_ylim(11.5, -.5)
        
fig.tight_layout(h_pad = 3., w_pad = 1.)

fig.savefig('./output/fig_2_T_confmats.pdf', bbox_inches='tight', dpi=200, transparent=True)

plt.show()

In [ ]:
for ibt, bt in enumerate(cm_bt) :
    for i, t in enumerate(cm_timesteps):
        p_mats = all_p_mats[ibt, i]
        signif = p_mats < 0.05
        n_diag = len(np.where(np.diag(signif) == True)[0])
        n_all = len(np.where(signif == True)[0])
        print('Bt=%.2f° ; t = %.2fs, \t signif pval diag = %s/%s' % (bt*180/np.pi, t,
                                              n_diag, 12))
    print('\n')

In [ ]:
for ibt, bt in enumerate(cm_bt) :
    for i, t in enumerate(cm_timesteps):
        cm = np.sum(all_cms[ibt, i])/all_cms[ibt,i].shape[0]
        signif = cm > 1/12
        n_diag = len(np.where(np.diag(signif) == True)[0])
        n_all = len(np.where(signif == True)[0])
        print('Bt=%.2f° ; t = %.2fs, \t %s / %s above chance level in diagonal (diag above chance level = %s/%s)' % (bt*180/np.pi, t,
                                             n_all , 12*12, n_diag, 12))
    print('\n')

# Temporal evolution

In [ ]:
try:
    all_accs = np.load('./data/fig_2_T_all_accs.npy')
    
except:
    all_accs = []
    for ibt, bt in enumerate(B_thetas) :
        # Data
        data, labels, le = par_load_temporal_data(timesteps = timesteps, target_btheta = bt,
                                                  target_theta = None, data_type = 'one_bt',
                                                  cluster_list = cluster_list)
 
        # Classifying
        logreg = LogisticRegression(**opts_LR)

        accs = []
        for ibin in range(data.shape[0]) :
            xtrain, xtest, ytrain, ytest = train_test_split(data[ibin,:,:], labels, test_size=test_size, random_state=42)
            logreg.fit(xtrain, ytrain)
            accs.append(metrics.balanced_accuracy_score(ytest, logreg.predict(xtest)))

        all_accs.append(accs)

    np.save('./data/fig_2_T_all_accs.npy', all_accs)
    


In [ ]:
fig, ax = plt.subplots(figsize = (8,5))

plot_bthetas = [0, 4, 7]
plot_bthetas = range(N_B_thetas)
colors = plt.cm.inferno(np.linspace(.8, .2, N_B_thetas))

for ibt in plot_bthetas :
    plot_data = np.asarray(all_accs[ibt])
    ax.plot(timesteps+win_size, plot_data, #color = colors[ibt, :], 
           label = r'B$_\theta$ = %.1f°' % (B_thetas[ibt] * 180/np.pi))
    
ax.hlines(.9, 0., .3, color = 'k')
ax.axhline(1/12, c = 'gray', linestyle = '--')

ax.set_xlabel('PST (s)')
ax.set_ylabel('Accuracy (%)')
ax.legend(loc = (.95, .7), frameon = False, fontsize = 12)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.show()

In [ ]:
B_thetas

# K-fold

In [ ]:
try:
    all_kfold_scores = np.load('./data/fig_2_T_all_kfold.npy')
    
except:
    all_kfold_scores = []
    for ibt, bt in enumerate(B_thetas) :
        # Data
        data, labels, le = par_load_temporal_data(timesteps = timesteps, target_btheta = bt,
                                                target_theta = None, data_type = 'one_bt',
                                                cluster_list = cluster_list)

        # Classifying
        logreg = LogisticRegression(**opts_LR)

        kfold_scores = [] 
        for ibin in range(data.shape[0]) :
            scores = cross_val_score(logreg, data[ibin,:,:], labels, 
                                     cv = n_splits, # number of folds
                                     scoring = 'balanced_accuracy')
            kfold_scores.append(scores)

        all_kfold_scores.append(kfold_scores)

    np.save('./data/fig_2_T_all_kfold.npy', all_kfold_scores)
    

In [ ]:
fig, ax = plt.subplots(figsize = (9,6))
colors = plt.cm.inferno(np.linspace(.8, .2, 8))

plot_bthetas = [7, 4, 0]
for ibt in plot_bthetas :
    
    kfold_means = np.asarray([x.mean() for x in all_kfold_scores[ibt]])
    kfold_stderr = np.asarray([x.std() for x in all_kfold_scores[ibt]])
    
    ax.plot(timesteps + win_size, kfold_means, color = colors[ibt])
    ax.fill_between(timesteps + win_size,
                kfold_means + kfold_stderr, kfold_means - kfold_stderr,
                facecolor = colors[ibt], edgecolor = None, alpha = .7,
                label = r'B$_\theta$ = %.1f°' % (B_thetas[ibt] * 180/np.pi))
    mod_t = timesteps + win_size
    print(ibt)
    print(mod_t[np.argmax(kfold_means)])
    
ax.hlines(.95, 0., .3, color = 'k', linewidth = 4)
ax.axhline(1/12, c = 'gray', linestyle = '--')
#ax.text(x = timesteps[-1] + .15, y = 1/len(le.classes_), s = 'Chance level', color = 'gray',
#       ha = 'center', va = 'center', fontsize = 12)

ax.set_xlabel('PST (s)', fontsize = 18)
ax.set_ylabel('Accuracy (%)', fontsize = 18)

ax.set_xlim(timesteps[0]+win_size, timesteps[-1]+win_size)
ax.set_ylim(0, 1.)

ax.legend(loc = (.025, .65), frameon = False, fontsize = 14, markerscale = .2)
leg = ax.get_legend()
leg.legendHandles[0].set_color(colors[7])
leg.legendHandles[0].set_alpha(1)
leg.legendHandles[1].set_color(colors[4])
leg.legendHandles[1].set_alpha(1)
leg.legendHandles[2].set_color(colors[0])
leg.legendHandles[2].set_alpha(1)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.tick_params(axis='both', which='major', labelsize=14)
labs = np.round(ax.get_yticks().tolist(),1)
ax.set_yticklabels((labs*100).astype(np.int16))

fig.savefig('./output/fig_2_decoder_timecourse.pdf', bbox_inches='tight', dpi=200, transparent=True)

plt.show()

# Optimization

In [ ]:
cols_subset = [-1, 4, 0]
B_thetas_subset = np.asarray([B_thetas[0], B_thetas[4], B_thetas[-1]])
B_thetas_subset = np.asarray([B_thetas[i] for i in cols_subset])

# Optim : Window size

In [ ]:
    
try:
    max_acc = np.load('./data/sup_5_T_opt_wsize.npy')
except:
    
    max_acc = []
    for win_size_ in win_sizes :
        
        mult_bt_acc = []
        for rand_state in np.arange(0, 4) :
            bt_acc = []
            for ibt, bt in enumerate(B_thetas_subset) :
                # Data
                data, labels, le = par_load_temporal_data(timesteps = timesteps, target_btheta = bt,
                                                        target_theta = None, data_type = 'one_bt',
                                                        cluster_list = cluster_list,
                                                        win_size=win_size_)

                # Classifying
                logreg = LogisticRegression(**opts_LR)

                accs = []
                for ibin in range(data.shape[0]) :
                    xtrain, xtest, ytrain, ytest = train_test_split(data[ibin,:,:], labels,
                                                                    test_size=test_size, random_state = 42 + rand_state,
                                                                    stratify = labels)
                    logreg.fit(xtrain, ytrain)
                    accs.append(metrics.balanced_accuracy_score(ytest, logreg.predict(xtest)))

                bt_acc.append(np.max(accs))
            mult_bt_acc.append(bt_acc)
        max_acc.append(mult_bt_acc)

    max_acc = np.array(max_acc)
    np.save('./data/sup_5_T_opt_wsize.npy', max_acc)


In [ ]:
max_acc = np.array(max_acc)
max_acc.shape

In [ ]:
fig, ax = plt.subplots(figsize = (1.61 * 7, 7))

lw = 3
ax.plot(win_sizes, max_acc[..., 0], color = colors[0],
       lw = lw, marker = 'o')
ax.plot(win_sizes, max_acc[..., 1], color = colors[4],
       lw = lw, marker = 'o')
ax.plot(win_sizes, max_acc[..., 2], color = colors[-1],
       lw = lw, marker = 'o')
ax.axhline(1/12, c = 'gray', linestyle = '--')

        
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

ax.set_xticks(win_sizes)
#ax.set_yticklabels([None, 40, 50, 60, 70, 80, 90])

ax.tick_params(axis='both', which='major', labelsize=14)
ax.set_xlabel('Decoder window size (s)', fontsize = 18)
ax.set_ylabel('Maximum decoding accuracy (%)', fontsize = 18)

fig.savefig('./output/sup_5_T_win_size.pdf', bbox_inches='tight', dpi=200, transparent=True)

plt.show()

# Optim : Penalty (L1,L2,mix(elasticnet))

In [ ]:
%rm ./data/sup_5_T_opt_penalty.npy

In [ ]:
try:
    max_acc = np.load('./data/sup_5_T_opt_penalty.npy')
except:
        
    max_acc = []
    for penalty in ['l1', 'l2', 'elasticnet']:
                
        bt_acc = []
        for ibt, bt in enumerate(B_thetas_subset) :
            data, labels, le = par_load_temporal_data(timesteps = timesteps, target_btheta = bt,
                                                    target_theta = None, data_type = 'one_bt',
                                                    cluster_list = cluster_list)

            # Classifying
            logreg = LogisticRegression(solver = 'lbfgs' if penalty == 'l2' else 'saga', 
                                        l1_ratio = .5 if penalty == 'elasticnet' else None, **opts_LR)

            accs = []
            for ibin in range(data.shape[0]) :
                xtrain, xtest, ytrain, ytest = train_test_split(data[ibin,:,:], labels, test_size=test_size, random_state = 42)
                logreg.fit(xtrain, ytrain)
                accs.append(metrics.balanced_accuracy_score(ytest, logreg.predict(xtest)))

            bt_acc.append(np.max(accs))
        max_acc.append(bt_acc)

    max_acc = np.array(max_acc)
    np.save('./data/sup_5_T_opt_penalty.npy', max_acc)
    


In [ ]:
fig, ax = plt.subplots(figsize = (1.61 * 7, 7))
width = .25

labs = ['L1', 'L2', 'ElasticNet(1:1)']

for i, bt in enumerate(B_thetas_subset) :
    ax.bar(x=i-width*1.1, height=max_acc[0][i],
           width=width,
           facecolor=colors[cols_subset[i]], edgecolor='k')
    ax.bar(x=i, height=max_acc[1][i],
           width=width,
           facecolor=colors[cols_subset[i]], edgecolor='k')
    ax.bar(x=i+width*1.1, height=max_acc[2][i],
           width=width,
           facecolor=colors[cols_subset[i]], edgecolor='k')
    
    y = .72
    ax.text(x=i-width*1.1, y=y,
            s='L1', rotation=45, ha='center', va='center', fontsize=12, c='w')
    ax.text(x=i, y=y,
            s='L2', rotation=45, ha='center', va='center', fontsize=12, c='w')
    ax.text(x=i+width*1.1, y=y,
            s='ElasticNet', rotation=45, ha='center', va='center', fontsize=12, c='w')
        
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

ax.set_xticklabels(np.round(B_thetas_subset*180/np.pi, 2)[::-1])
ax.set_xticks([2, 1, 0][::-1])

#ax.set_ylim(.65, .92)


ax.tick_params(axis='both', which='major', labelsize=14)

ax.set_xlabel(r'$B_\theta$(°)', fontsize = 18)
ax.set_ylabel('Maximum decoding accuracy (%)', fontsize = 18)

fig.savefig('./output/sup_5_T_penalty.pdf', bbox_inches='tight', dpi=200, transparent=True)

plt.show()

# Optim : C = 1/$\lambda$ (.01 to 10)

In [ ]:
try:
    max_acc = np.load('./data/sup_5_T_opt_C.npy')
except:

    max_acc = []
    for C_ in Cs:
        
        bt_acc = []
        for ibt, bt in enumerate(B_thetas_subset) :

            data, labels, le = par_load_temporal_data(timesteps = timesteps, target_btheta = bt,
                                                      target_theta = None, data_type = 'one_bt',
                                                      cluster_list = cluster_list)

            # Classifying
            logreg = LogisticRegression(**opts_LR.update(C = C_)
                                        )

            accs = []
            for ibin in range(data.shape[0]) :
                xtrain, xtest, ytrain, ytest = train_test_split(data[ibin,:,:], labels, test_size=test_size, random_state = 42)
                logreg.fit(xtrain, ytrain)
                accs.append(metrics.balanced_accuracy_score(ytest, logreg.predict(xtest)))

            bt_acc.append(np.max(accs))
        max_acc.append(bt_acc)

    max_acc = np.array(max_acc)
    np.save('./data/sup_5_T_opt_C.npy', max_acc)
    


In [ ]:
fig, ax = plt.subplots(figsize = (1.61 * 7, 7))
width = .25

labs = ['L1', 'L2', 'ElasticNet(1:1)']

for i, bt in enumerate(np.linspace(.01, 3, 10)) :
    for col in cols_subset:
        ax.bar(x=i-width*1.1, height=max_acc[i, 0],
               width=width,
               facecolor=colors[cols_subset[-1]], edgecolor='k')
    ax.bar(x=i, height=max_acc[i, 1],
           width=width,
           facecolor=colors[cols_subset[1]], edgecolor='k')
    ax.bar(x=i+width*1.1, height=max_acc[i, 2],
           width=width,
           facecolor=colors[cols_subset[0]], edgecolor='k')
    
    if i == 0 :
        ax.text(x=i-width, y=.9,
                s=r'$B_\theta$=%.2f°' % (B_thetas_subset[0] * 180/np.pi),
                rotation=90, ha='center', va='center', fontsize=12, c='k')
        ax.text(x=i, y=.9,
                s=r'$B_\theta$=%.2f°' % (B_thetas_subset[1] * 180/np.pi),
                rotation=90, ha='center', va='center', fontsize=12, c='k')
        ax.text(x=i+width, y=.9,
                s=r'$B_\theta$=%.2f°' % (B_thetas_subset[2] * 180/np.pi),
                rotation=90, ha='center', va='center', fontsize=12, c='k')
        
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

ax.set_xticks(np.arange(0, 10, 1))
ax.set_xticklabels(np.round(np.linspace(.01, 3, 10), 2))

ax.set_ylim(.5, .9)
ax.set_yticklabels([None, 50, 55, 60, 65, 70, 75, 80, 85, 90])

ax.tick_params(axis='both', which='major', labelsize=14)

ax.set_xlabel('C', fontsize = 18)
ax.set_ylabel('Maximum decoding accuracy (%)', fontsize = 18)

fig.savefig('./output/sup_5_T_C.pdf', bbox_inches='tight', dpi=200, transparent=True)

plt.show()

# Optim : Train/test size (.15, .5)

In [ ]:
try:
    max_acc = np.load('./data/sup_5_T_opt_size.npy')

except:
        
    max_acc = []
    for test_size_ in test_sizes:

                
        bt_acc = []
        for ibt, bt in enumerate(B_thetas_subset) :
            

            data, labels, le = par_load_temporal_data(timesteps = timesteps, target_btheta = bt,
                                                      target_theta = None, data_type = 'one_bt',
                                                      cluster_list = cluster_list)
            
            # Classifying
            print(opts_LR) # HACK to check C is not changed
            logreg = LogisticRegression(**opts_LR)

            accs = []
            for ibin in range(data.shape[0]) :
                xtrain, xtest, ytrain, ytest = train_test_split(data[ibin,:,:], labels, test_size=test_size_, random_state = 42)
                logreg.fit(xtrain, ytrain)
                accs.append(metrics.balanced_accuracy_score(ytest, logreg.predict(xtest)))

            bt_acc.append(np.max(accs))
        max_acc.append(bt_acc)

    max_acc = np.array(max_acc)
    np.save('./data/sup_5_T_opt_size.npy', max_acc)
    


In [ ]:
fig, ax = plt.subplots(figsize = (1.61 * 7, 7))
width = .25

labs = ['L1', 'L2', 'ElasticNet(1:1)']

cols = [-1, 4, 0]
for i, _ in enumerate(test_sizes) :
    ax.bar(x=i-width, height=max_acc[i, 0],
           width=width,
           facecolor=colors[cols_subset[-1]], edgecolor='k')
    ax.bar(x=i, height=max_acc[i, 1],
           width=width,
           facecolor=colors[cols_subset[1]], edgecolor='k')
    ax.bar(x=i+width, height=max_acc[i, 2],
           width=width,
           facecolor=colors[cols_subset[0]], edgecolor='k')
    
    if i == 0 :
        ax.text(x=i-width, y=.88,
                s=r'$B_\theta$=%.2f°' % (B_thetas_subset[0] * 180/np.pi),
                rotation=90, ha='center', va='center', fontsize=12, c='k')
        ax.text(x=i, y=.88,
                s=r'$B_\theta$=%.2f°' % (B_thetas_subset[1] * 180/np.pi),
                rotation=90, ha='center', va='center', fontsize=12, c='k')
        ax.text(x=i+width, y=.88,
                s=r'$B_\theta$=%.2f°' % (B_thetas_subset[2] * 180/np.pi),
                rotation=90, ha='center', va='center', fontsize=12, c='k')
        
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

ax.set_xticks(np.arange(0, 6, 1))
ax.set_xticklabels(np.int16(np.round(np.linspace(.15, .5, 6)*100)))

ax.set_ylim(.70, .875)
#ax.set_yticklabels([None, 72, 74, 76, 78, 80, 82, 84, 86])

ax.tick_params(axis='both', which='major', labelsize=14)

ax.set_xlabel('Test set size (% of train set size)', fontsize = 18)
ax.set_ylabel('Maximum decoding accuracy (%)', fontsize = 18)

fig.savefig('./output/sup_5_T_size.pdf', bbox_inches='tight', dpi=200, transparent=True)

plt.show()